In [1]:
from toolz import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
%matplotlib widget


import pathlib
from lenses import lens

from collections import Counter

import re
import nltk
import pymorphy2
import fasttext.util

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
nltk.download('popular')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('tagsets')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to

True

In [3]:
lmap = compose(list, map)
ltake = compose(list, take)

In [4]:
def map_df_ingredients_element(func, df):
    df = df.copy()
    df['ingredients'] = df['ingredients'].map(lambda v: lmap(func, v))
    return df

In [5]:
data_path = pathlib.Path('data')

## Загружаем датафрейм с данными

In [6]:
df = pd.read_json(data_path.joinpath('ready_dataframe.json'))
df = map_df_ingredients_element(compose(str.lower, str.strip), df)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"[бананы, лимонный сок, молотая корица, ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"[сахар, кокосовое молоко, корень имбиря, анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"[клубника, манго, сахар, тертая цедра лайма, т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"[манго, темный ром, сок лайма, сахар, тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"[клубника, сахарная пудра, сок лайма, текила, ..."
...,...,...,...,...,...
41433,43380,Постный борщ с фасолью,Супы,Украинская кухня,"[белая фасоль, овощной бульон, свекла, картофе..."
41434,80446,Суп энгамат,Супы,Шведская кухня,"[цветная капуста, морковь, картофель, лук-поре..."
41435,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"[горох, репчатый лук, гвоздика, свиная рулька,..."
41436,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"[оливковое масло, репчатый лук, овощной бульон..."


## Работа с ингридиентами

Удобнее работать с листом листов:

In [7]:
ingredients = list(df['ingredients'])
# TODO !!!!!!!!! Временно берем только первые несколько записей 
ingredients = ingredients 
ingredients[:3]

[['бананы', 'лимонный сок', 'молотая корица', 'ванильное мороженое'],
 ['сахар', 'кокосовое молоко', 'корень имбиря', 'ананас', 'сок лайма'],
 ['клубника',
  'манго',
  'сахар',
  'тертая цедра лайма',
  'текила',
  'сок лайма',
  'апельсиновый ликер']]

### Самые часто встречающиеся слова в ингридиентах

In [8]:
ingredients_counter = Counter()
for lst in ingredients:
    ingredients_counter.update(lst)
# first_ingresients, _ = zip(*ingredients_counter.most_common(10))
# first_ingresients = list(first_ingresients)
# first_ingresients
ingredients_counter.most_common(10)

[('соль', 20427),
 ('куриное яйцо', 11621),
 ('сахар', 10953),
 ('молотый черный перец', 10877),
 ('сливочное масло', 10323),
 ('пшеничная мука', 9950),
 ('чеснок', 9558),
 ('оливковое масло', 7932),
 ('репчатый лук', 7272),
 ('растительное масло', 6456)]

In [9]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(word):
    return morph.parse(word)[0].normal_form

it = lmap(compose(tuple, nltk.word_tokenize), ingredients_counter)
it = lens.Each().Each().modify(lemmatize)(it)
it[:10]

[('банан',),
 ('лимонный', 'сок'),
 ('молотый', 'корица'),
 ('ванильный', 'мороженое'),
 ('сахар',),
 ('кокосовый', 'молоко'),
 ('корень', 'имбирь'),
 ('ананас',),
 ('сок', 'лайм'),
 ('клубника',)]

### Токенизация

In [10]:
ingredients_tokenized = lens.Each().Each().modify(nltk.word_tokenize)(ingredients)
ingredients_tokenized[:4]

[[['бананы'],
  ['лимонный', 'сок'],
  ['молотая', 'корица'],
  ['ванильное', 'мороженое']],
 [['сахар'],
  ['кокосовое', 'молоко'],
  ['корень', 'имбиря'],
  ['ананас'],
  ['сок', 'лайма']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['текила'],
  ['сок', 'лайма'],
  ['апельсиновый', 'ликер']],
 [['манго'],
  ['темный', 'ром'],
  ['сок', 'лайма'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['ананас']]]

### Стоп-слова и фильтрация

In [11]:
stopwords = nltk.corpus.stopwords.words('russian')

In [12]:
rx = re.compile(r'[a-zA-Zа-яА-Я][a-zA-Zа-яА-Я\d\-]*')

In [13]:
ingredients_filtered = (lens
    .Each().Each().modify(
        lambda tokenized: tuple(filter(
            lambda word: word not in stopwords and rx.match(word),
            tokenized))
    )(ingredients_tokenized)
)
ingredients_filtered[:4]

[[('бананы',),
  ('лимонный', 'сок'),
  ('молотая', 'корица'),
  ('ванильное', 'мороженое')],
 [('сахар',),
  ('кокосовое', 'молоко'),
  ('корень', 'имбиря'),
  ('ананас',),
  ('сок', 'лайма')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('текила',),
  ('сок', 'лайма'),
  ('апельсиновый', 'ликер')],
 [('манго',),
  ('темный', 'ром'),
  ('сок', 'лайма'),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('ананас',)]]

### Лемматизация

In [14]:
ingredients_lemmatized = lens.Each().Each().Each().modify(lemmatize)(ingredients_filtered)
ingredients_lemmatized[:4]

[[('банан',),
  ('лимонный', 'сок'),
  ('молотый', 'корица'),
  ('ванильный', 'мороженое')],
 [('сахар',),
  ('кокосовый', 'молоко'),
  ('корень', 'имбирь'),
  ('ананас',),
  ('сок', 'лайм')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('текила',),
  ('сок', 'лайм'),
  ('апельсиновый', 'ликёр')],
 [('манго',),
  ('тёмный', 'ром'),
  ('сок', 'лайм'),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('ананас',)]]

In [15]:
all_words_lemmatized = Counter(lens.Each().Each().Each().collect()(ingredients_lemmatized))

all_words_lemmatized_list = list(all_words_lemmatized)

#Добавил UNK и PAD
all_words_lemmatized_list = ['PAD', 'UNK']+all_words_lemmatized_list
all_words_lemmatized_list.sort()
all_words_lemmatized_list[:10]
# dict(all_words.items())

['PAD',
 'UNK',
 'añejo',
 'baby',
 'baileys',
 'bambino',
 'bean',
 'biogourmet',
 'biscoff',
 'blue']

In [16]:
#ОЧЕНЬ ДОЛГО
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [17]:
ft = fasttext.load_model('cc.ru.300.bin')
dim = ft.get_dimension()
dim

300

### Получаем векторы для слов 

In [18]:
word_embeddings_dict = {}
for word in all_words_lemmatized_list:
    if word  == 'PAD':
        word_embeddings_dict[word] = np.concatenate((np.zeros(dim),[1, 0]))
    elif word  == 'UNK':
        word_embeddings_dict[word] = np.concatenate((np.zeros(dim),[0, 1]))
    else:
        word_embeddings_dict[word] = np.concatenate((ft.get_word_vector(word), [0, 0]))

In [19]:
first(word_embeddings_dict.items())
# np.concatenate((ft.get_word_vector(word), [0, 0])).shape

('PAD',
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

Расширяем разметрости для PAD и UNK

In [20]:
dim += 2

In [21]:
del ft

In [22]:
# word_num = [(word, i) for i, word in enumerate(words)]
# word_num[:10]

### Векторы для ингредиентов

In [23]:
all_ingredients_list =  list(set(lens.Each().Each().collect()(ingredients_lemmatized)))
#Добавил UNK и PAD
all_ingredients_list = [('PAD',), ('UNK',)]+all_ingredients_list
all_ingredients_list 
ingredients_numbering_dict = {ingredient: i for i, ingredient in enumerate(all_ingredients_list)}
ltake(6, ingredients_numbering_dict.items())

[(('PAD',), 0),
 (('UNK',), 1),
 (('кукурузный', 'масло', 'слобода'), 2),
 (('петух',), 3),
 (('жир', 'растительный'), 4),
 (('варёный', 'окорка'), 5)]

Для получения вектора для ингридиента просто суммируем векторы для каждого слова в этом ингридиенте:

In [24]:
ingredients_embeddings_dict = {}
for ingr in all_ingredients_list:
    ingredients_embeddings_dict[ingr] = np.zeros(dim)
    for word in ingr:
#         ingredients_embeddings_dict[ingr] += ft.get_word_vector(word) 
        ingredients_embeddings_dict[ingr] += word_embeddings_dict[word] # Должно заработать, когда 
                                                                         # ингредиенты будут лемматизированны
ltake(1, ingredients_embeddings_dict.items())[0]


(('PAD',),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
 

Наконец, для каждого рецепта возьмем его векторизацию как сумму векторов его ингридиентов

In [25]:
ingredients_by_recipe_embedding = [
    sum(ingredients_embeddings_dict[ingredient]
        for ingredient in ingredient_list) 
    for ingredient_list in ingredients_lemmatized
]
len(ingredients_by_recipe_embedding)

41438

## Данные для нейросети

Всевозможные метки (course):

In [26]:
all_courses = {name: num for num, name in enumerate(df['course'][~df['course'].duplicated()])}
all_courses

{'Выпечка и десерты': 0,
 'Завтраки': 1,
 'Закуски': 2,
 'Напитки': 3,
 'Основные блюда': 4,
 'Паста и пицца': 5,
 'Салаты': 6,
 'Соусы и маринады': 7,
 'Супы': 8}

In [27]:
df_for_learning = pd.DataFrame(data={
    'id': df['id'],
    'ingredient_embedding': ingredients_by_recipe_embedding,
    'course_mark': [all_courses[course] for course in df['course']]
})
df_for_learning

,id,ingredient_embedding,course_mark
0,28195,"[0.611611146479845, -0.0743011748418212, -0.09...",0
1,28141,"[0.8909059994039126, -0.11706455610692501, -0....",0
2,27929,"[1.08976399153471, -0.4261779775843024, -0.389...",0
3,28192,"[1.1749160811305046, -0.5175759345293045, -0.3...",0
4,28463,"[0.6190221486613154, -0.14342674519866705, -0....",0
...,...,...,...
41433,43380,"[1.3148583127185702, -0.05140630202367902, -0....",8
41434,80446,"[1.0693077351897955, 0.24955729965586215, -0.1...",8
41435,136820,"[0.7934096870012581, 0.5892392117530107, -0.44...",8
41436,18014,"[0.834478304721415, 0.4054728071205318, 0.2355...",8


In [28]:
# tf.keras.preprocessing.text.one_hot(all_courses, len(all_courses))

## Подготовка BoE

In [29]:
data = np.array(ingredients_by_recipe_embedding)
labels = to_categorical(np.array(df_for_learning['course_mark']), num_classes=len(all_courses))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

## Обучение BoE

In [30]:
sequence_input = layers.Input(shape=(dim,), dtype='int32')
x = layers.Dense(64, activation='relu')(sequence_input)
x = layers.Dense(16, activation='relu')(x)
preds = layers.Dense(len(all_courses), activation='softmax')(x)

model = keras.Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])


In [31]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 302)]             0         
                                                                 
 dense (Dense)               (None, 64)                19392     
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                                 
Total params: 20,585
Trainable params: 20,585
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=20, batch_size=128)

Epoch 1/200
259/259 [==============================] - 2s 3ms/step - loss: 1.6185 - acc: 0.4564 - val_loss: 1.4407 - val_acc: 0.5005
Epoch 2/200
259/259 [==============================] - 1s 2ms/step - loss: 1.4047 - acc: 0.5063 - val_loss: 1.3752 - val_acc: 0.5172
Epoch 3/200
259/259 [==============================] - 1s 2ms/step - loss: 1.3538 - acc: 0.5210 - val_loss: 1.3589 - val_acc: 0.5189
Epoch 4/200
259/259 [==============================] - 1s 2ms/step - loss: 1.3285 - acc: 0.5260 - val_loss: 1.3360 - val_acc: 0.5272
Epoch 5/200
259/259 [==============================] - 1s 2ms/step - loss: 1.3119 - acc: 0.5309 - val_loss: 1.3292 - val_acc: 0.5258
Epoch 6/200
259/259 [==============================] - 1s 2ms/step - loss: 1.2984 - acc: 0.5361 - val_loss: 1.3183 - val_acc: 0.5291
Epoch 7/200
259/259 [==============================] - 1s 2ms/step - loss: 1.2874 - acc: 0.5370 - val_loss: 1.3217 - val_acc: 0.5282
Epoch 8/200
259/259 [==============================] - 1s 2ms/step - 

259/259 [==============================] - 1s 2ms/step - loss: 1.0458 - acc: 0.6284 - val_loss: 1.4930 - val_acc: 0.5126
Epoch 63/200
259/259 [==============================] - 1s 2ms/step - loss: 1.0422 - acc: 0.6293 - val_loss: 1.5121 - val_acc: 0.5202
Epoch 64/200
259/259 [==============================] - 1s 2ms/step - loss: 1.0408 - acc: 0.6280 - val_loss: 1.5371 - val_acc: 0.5005
Epoch 65/200
259/259 [==============================] - 1s 2ms/step - loss: 1.0391 - acc: 0.6300 - val_loss: 1.5009 - val_acc: 0.5154
Epoch 66/200
259/259 [==============================] - 1s 3ms/step - loss: 1.0360 - acc: 0.6309 - val_loss: 1.5393 - val_acc: 0.5164
Epoch 67/200
259/259 [==============================] - 1s 2ms/step - loss: 1.0335 - acc: 0.6329 - val_loss: 1.5224 - val_acc: 0.5096
Epoch 68/200
259/259 [==============================] - 1s 2ms/step - loss: 1.0311 - acc: 0.6323 - val_loss: 1.5312 - val_acc: 0.5179
Epoch 69/200
259/259 [==============================] - 1s 2ms/step - loss:

259/259 [==============================] - 1s 2ms/step - loss: 0.9493 - acc: 0.6632 - val_loss: 1.8086 - val_acc: 0.5077
Epoch 124/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9485 - acc: 0.6619 - val_loss: 1.8246 - val_acc: 0.5010
Epoch 125/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9460 - acc: 0.6633 - val_loss: 1.8326 - val_acc: 0.5051
Epoch 126/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9452 - acc: 0.6634 - val_loss: 1.8417 - val_acc: 0.5010
Epoch 127/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9440 - acc: 0.6658 - val_loss: 1.8420 - val_acc: 0.4995
Epoch 128/200
259/259 [==============================] - 1s 3ms/step - loss: 0.9431 - acc: 0.6657 - val_loss: 1.8556 - val_acc: 0.5072
Epoch 129/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9440 - acc: 0.6662 - val_loss: 1.8522 - val_acc: 0.5034
Epoch 130/200
259/259 [==============================] - 1s 2ms/step 

Epoch 184/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9033 - acc: 0.6793 - val_loss: 2.1362 - val_acc: 0.4940
Epoch 185/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9025 - acc: 0.6795 - val_loss: 2.1083 - val_acc: 0.4967
Epoch 186/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9032 - acc: 0.6800 - val_loss: 2.1219 - val_acc: 0.4946
Epoch 187/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9011 - acc: 0.6792 - val_loss: 2.1384 - val_acc: 0.4834
Epoch 188/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9017 - acc: 0.6804 - val_loss: 2.1255 - val_acc: 0.4900
Epoch 189/200
259/259 [==============================] - 1s 2ms/step - loss: 0.8999 - acc: 0.6815 - val_loss: 2.1067 - val_acc: 0.4882
Epoch 190/200
259/259 [==============================] - 1s 2ms/step - loss: 0.9004 - acc: 0.6808 - val_loss: 2.1316 - val_acc: 0.4892
Epoch 191/200
259/259 [==============================] 

## Подготовка к CNN

In [33]:
data = lens.Each().Each().modify(lambda x: ingredients_numbering_dict[x])(ingredients_lemmatized)

In [34]:
data = pad_sequences(data, padding='post', value=0)
labels = to_categorical(np.array(df_for_learning['course_mark']), num_classes=len(all_courses))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [35]:
max_length = len(data[0])
max_length

40

In [36]:
#Эта матрица будет использоваться для слоя Embedding в keras
ingredients_embeddings_matrix = np.zeros((len(all_ingredients_list), dim))
ingredients_embeddings_matrix.shape
for ingr in all_ingredients_list:
    ingredients_embeddings_matrix[ingredients_numbering_dict[ingr]] = ingredients_embeddings_dict[ingr]
ingredients_embeddings_matrix.shape

(4406, 302)

## Обучение CNN

In [37]:
embedding_layer = layers.Embedding(len(ingredients_embeddings_dict),
                            dim,
                            weights=[ingredients_embeddings_matrix],
                            input_length=max_length,
                            trainable=False)
# trainable=True позволит до обучаться эмбеддингам на нашем датасете

In [38]:
sequence_input = layers.Input(shape=(max_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
preds = layers.Dense(len(all_courses), activation='softmax')(x)

model = keras.Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])



In [39]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40)]              0         
                                                                 
 embedding (Embedding)       (None, 40, 302)           1330612   
                                                                 
 conv1d (Conv1D)             (None, 36, 128)           193408    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 128)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 896)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               114816    
                                                           

In [40]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=20, batch_size=128)

Epoch 1/50
259/259 [==============================] - 7s 25ms/step - loss: 1.2437 - acc: 0.5609 - val_loss: 1.0206 - val_acc: 0.6239
Epoch 2/50
259/259 [==============================] - 6s 23ms/step - loss: 0.9581 - acc: 0.6579 - val_loss: 0.9434 - val_acc: 0.6573
Epoch 3/50
259/259 [==============================] - 7s 27ms/step - loss: 0.8534 - acc: 0.6939 - val_loss: 0.8541 - val_acc: 0.6948
Epoch 4/50
259/259 [==============================] - 7s 26ms/step - loss: 0.7816 - acc: 0.7184 - val_loss: 0.8729 - val_acc: 0.6943
Epoch 5/50
259/259 [==============================] - 8s 30ms/step - loss: 0.7259 - acc: 0.7385 - val_loss: 0.8559 - val_acc: 0.6960
Epoch 6/50
259/259 [==============================] - 7s 26ms/step - loss: 0.6767 - acc: 0.7555 - val_loss: 0.8308 - val_acc: 0.7082
Epoch 7/50
259/259 [==============================] - 7s 28ms/step - loss: 0.6340 - acc: 0.7745 - val_loss: 0.8930 - val_acc: 0.6889
Epoch 8/50
259/259 [==============================] - 7s 27ms/step - 

KeyboardInterrupt: 

## Прочее:

In [ ]:
it = df.groupby('cuisine').count()
it.sort_values('title', ascending=False).head(30)

In [ ]:
a = ft.get_word_vector('кокосовый')-ft.get_word_vector('кокосовое')+ft.get_word_vector('кремовое')-ft.get_word_vector('кремовый')
a@a